# Standardize and Correct Data Spelling

First, let's import some useful libraries. These are ready-made sets of commands which can do tasks; in our case, we want tasks such as reading CSV files, merging them, summarizing them, etc. We will mainly use **pandas**.

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import glob
import os

In [2]:
#df1['State'] = df1['State'].str.lstrip("0")

In [3]:
cd C:\Users\dakot\Documents\Trees\Data Cleaning\/Sheets_to_Clean

C:\Users\dakot\Documents\Trees\Data Cleaning\Sheets_to_Clean


In [4]:
csv_files = glob.glob('*.csv')
print("I will edit the following " + str(len(csv_files))+ " files")
print(csv_files)

I will edit the following 67 files
['Albuquerque_trees.csv', 'Anaheim_trees.csv', 'Arlington_trees.csv', 'Atlanta_trees.csv', 'AuroraCO_trees.csv', 'Austin_trees.csv', 'Baltimore_trees.csv', 'Boston_trees.csv', 'Buffalo_trees.csv', 'CapeCoral_trees.csv', 'ColoradoSprings_trees.csv', 'Columbus_trees.csv', 'Dallas_trees.csv', 'Denver_trees.csv', 'DesMoines_trees.csv', 'Detroit_tree.csv', 'Durham_trees.csv', 'ElPasoPERCENT_trees.csv', 'Fresno_trees.csv', 'GardenGrove_trees.csv', 'GrandRapids_trees.csv', 'Greensboro_trees.csv', 'Honolulu_trees.csv', 'Houston_trees.csv', 'HuntingtonBeach_trees.csv', 'Indianapolis_trees.csv', 'Irvine_trees.csv', 'Jerseycity_trees.csv', 'Knoxville_trees.csv', 'LasVegas_trees.csv', 'LosAngeles1_trees.csv', 'LosAngeles2_trees.csv', 'LosAngeles3_trees.csv', 'LosAngeles4_trees.csv', 'Louisville_trees.csv', 'Madison_trees.csv', 'Miami_trees.csv', 'Milwaukee_trees.csv', 'Minneapolis_trees.csv', 'Nashville_trees.csv', 'NewOrleans_trees.csv', 'NewYork_trees.csv', 'Oa

## edit all important columns

#### scientific names
- strip leading space (df1['State'] = df1['State'].str.strip())
- replace ' X ' with ' ' df.replace([0, 1, 2, 3], [4, 3, 2, 1])
- replace ' x ' with ' '
- remove anyything in quotes 'darlingtoni'
- Take first two words, make capitalization , Genus species 
- replace with "" the wide variety of non-tree answers
- replace a few very common name typos, caught manually
#### common names:
- fix cases wher it is split by a comma
- remove non ascii characters
- fix typos
- replace spp etc with ''
- capitalize first word
#### Address Issues       
- fix capitalizations for addresses
- combine street name and street number
- if no street number, just keep street name as the address
#### Overhead Utility       
- categorize as yes or no based on the variety of ways cities code it
#### Columns that just need simple capitalization fixes, etc
- 'location_type','location_name','neighborhood','district','planter_type','ward'
- remove non ASCII characters
- strip spaces
- capitalize
#### Height
- convert FT columns to meters
#### Height Range
- standardize the height ranges presented to us
#### Bin Heights
- for analyses where we bin heights, to include as many cities as possible
- sort into bins
#### Diameter Breast Height
- convert IN to CM
#### Diameter Breast Height Range
- standardize 
#### Diameter Breast Height Binned
- for analyses where we bin diameters, to include as many cities as possible
- sort into bins
#### Condition
- standardize the many ways cities code condition
- note that we checked with specific city metadata where needed
#### Date Columns
- cities recorded a wide variety of dates
- cycle through all date like columsn adn standardize them
- take the most recent observation out of 'inventory_date', 'condition_date','edit_date', 'inspected_date' and save it


In [5]:
# set path for where to save edited files
path=r'C:\Users\dakot\Documents\Trees\Data Cleaning\Sheets_Data_Misspellings_Corrected'

for file in csv_files:
#for file in csv_files[25:26]:
#for file in csv_files[26:56]:    
    data = pd.read_csv(file)
    # save the state name
    state = file.split('_')[0]
    #if it has scientific names
#############################
###  Scientific Name      
#############################
    if 'scientific_name' in data.columns:
        data['scientific_name_old']=data['scientific_name']
        ## quick fix to capitalization issues
        data['scientific_name']=data['scientific_name'].str.capitalize()
        ## fix issues
        data['scientific_name']=data['scientific_name'].str.strip()
        data['scientific_name'] = data['scientific_name'].str.replace(" X ", " ")
        data['scientific_name'] = data['scientific_name'].str.replace("X ", "")
        data['scientific_name'] = data['scientific_name'].str.replace("Ã—","")
        data['scientific_name'] = data['scientific_name'].str.replace("Â","")
        data['scientific_name'] = data['scientific_name'].str.replace(" x ", " ")
        data['scientific_name'] = data['scientific_name'].str.replace(" x$", "")
        data['scientific_name'] = data['scientific_name'].str.replace("::", " ")
        data['scientific_name'] = data['scientific_name'].str.replace(r'\'.*?\'', '')   
        data['scientific_name'] = data['scientific_name'].str.replace('*', '')  
        data['scientific_name'] = data['scientific_name'].str.replace('?', '') 
        # now get rid of vacant sites, stumps, etc.
        #data['scientific_name'] = data['scientific_name'].str.replace(r'Vacant.*','')
        data['scientific_name'] = data['scientific_name'].str.replace(r'Stump.*', '')
        data['scientific_name'] = data['scientific_name'].str.replace('Stump-', '')
        data['scientific_name'] = data['scientific_name'].str.replace('Standing butt', '')
        data['scientific_name'] = data['scientific_name'].str.replace(r'Unknown.*', '')
        data['scientific_name'] = data['scientific_name'].str.replace(r'Other.*', '')
        # formerly replaced all these with ' sp.'
        data['scientific_name'] = data['scientific_name'].str.replace(" spp\.", '')
        data['scientific_name'] = data['scientific_name'].str.replace(" spp", '')
        data['scientific_name'] = data['scientific_name'].str.replace(" species", '')
        # data['scientific_name'] = data['scientific_name'].str.replace(r' sp$','')
        data['scientific_name'] = data['scientific_name'].str.replace(' sp\.','')
        ##########
        ## remove non ASCII characters
        data['scientific_name'] = data['scientific_name'].str.encode('ascii', 'ignore').str.decode('ascii')
        ## get two name format, remove subspecies
        names = data['scientific_name'].str.split(expand=True)[[0,1]]
        names[1].replace(to_replace=[None], value='', inplace=True)
        data['scientific_name'] = names[0].str.lower().str.capitalize() + ' ' + names[1].str.lower()
        data['scientific_name'] = data['scientific_name'].str.strip()
        ########## REPLACE a bunch of manual typos
        data['scientific_name'] = data['scientific_name'].replace(['Rock'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Quercus/live virginiana'], 'Quercus virginiana')
        data['scientific_name'] = data['scientific_name'].replace(['Mixed'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Unidentifiable tree'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Mixed brush'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Hardwood'], '')
        data['scientific_name'] = data['scientific_name'].replace(['#name'], '')
        data['scientific_name'] = data['scientific_name'].replace([')amelanchier canadensis'], 'Amelanchier canadensis')
        data['scientific_name'] = data['scientific_name'].replace([')carpinus caroliniana'], 'Carpinus caroliniana')
        data['scientific_name'] = data['scientific_name'].replace(['0'], '')
        data['scientific_name'] = data['scientific_name'].replace(['10:15:54'], '')
        data['scientific_name'] = data['scientific_name'].replace(['15948'], '')
        data['scientific_name'] = data['scientific_name'].replace(['443'], '')
        data['scientific_name'] = data['scientific_name'].replace(['444'], '')
        data['scientific_name'] = data['scientific_name'].replace(['445'], '')
        data['scientific_name'] = data['scientific_name'].replace(['446'], '')
        data['scientific_name'] = data['scientific_name'].replace(['448'], '')
        data['scientific_name'] = data['scientific_name'].replace(['A. arnoldiana\'autmn'], 'Aesculus arnoldiana')
        data['scientific_name'] = data['scientific_name'].replace(['Acre rubrum'], 'Acer rubrum')
        data['scientific_name'] = data['scientific_name'].replace(['Acre saccharinum'], 'Acer saccharum')
        data['scientific_name'] = data['scientific_name'].replace(['Alder'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Almond tree'], 'Prunus dulcis')
        data['scientific_name'] = data['scientific_name'].replace(['Angohpora angophora'], 'Angophora ')
        data['scientific_name'] = data['scientific_name'].replace(['AngophoraÂ '], 'Angophora')
        data['scientific_name'] = data['scientific_name'].replace(['Apricot tree'], 'Prunus')
        data['scientific_name'] = data['scientific_name'].replace(['Arborvitae sp'], 'Thuja')
        data['scientific_name'] = data['scientific_name'].replace(['Arecacea'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Arecaecea'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Ash'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Banana'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Bonsai'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Broadleaf deciduous'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Broadleaf evergreen'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Calls'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Camaldulensis'], 'Eucalyptus camaldulensis')
        data['scientific_name'] = data['scientific_name'].replace(['Cedric deodora'], 'Cedrus deodara')
        data['scientific_name'] = data['scientific_name'].replace(['Cherry/prunus'], 'Prunus')
        data['scientific_name'] = data['scientific_name'].replace(['Cherry\/prunus'], 'Prunus sp.')
        data['scientific_name'] = data['scientific_name'].replace(['Chitalpatashkentensis'], 'Chitalpa tashkentensis')
        data['scientific_name'] = data['scientific_name'].replace(['Clerics canadensis'], 'Cercis canadensis')
        data['scientific_name'] = data['scientific_name'].replace(['Conifer evergreen'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Cunnighamia'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Ddigs tree'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Dead tree'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Dead'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Empty pit'], '')
        data['scientific_name'] = data['scientific_name'].replace(['EucalyptusÂ camaldulensis'], 'Eucalyptus camaldulensis')
        data['scientific_name'] = data['scientific_name'].replace(['Eucommiaulmoides'], 'Eucommia ulmoides')
        data['scientific_name'] = data['scientific_name'].replace(['Fig tree'], 'Ficus sp.')
        data['scientific_name'] = data['scientific_name'].replace(['For unknown'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Forthysia'], 'Forsythia')
        data['scientific_name'] = data['scientific_name'].replace(['Fruit tree'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Guava'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Holly, fosters'], 'Ilex attenuata')
        data['scientific_name'] = data['scientific_name'].replace(['Lageoemia indica'], 'Lagerstroemia indica')
        data['scientific_name'] = data['scientific_name'].replace(['Lageoemiasarahsfavorite'], 'Lageoemia indica')
        data['scientific_name'] = data['scientific_name'].replace(['Lagerstroemiasarahsfavorite'], 'Lagerstroemia indica')
        data['scientific_name'] = data['scientific_name'].replace(['Lagertsroemia indicafauriei'], 'Lagerstroemia indica')
        data['scientific_name'] = data['scientific_name'].replace(['LigumÂ '], '')
        data['scientific_name'] = data['scientific_name'].replace(['Linden'], 'Tilia sp.')
        data['scientific_name'] = data['scientific_name'].replace(['Locust'], '')
        data['scientific_name'] = data['scientific_name'].replace(['London plane'], 'Platanus acerifolia')
        data['scientific_name'] = data['scientific_name'].replace(['Longleaf pine'], 'Pinus palis')
        data['scientific_name'] = data['scientific_name'].replace(['Magnolialiliflora liliflora'], 'Magnolia liliiflora')
        data['scientific_name'] = data['scientific_name'].replace(['Malcura pomnifera'], 'Maclura pomifera')
        data['scientific_name'] = data['scientific_name'].replace(['No replant'], '')
        data['scientific_name'] = data['scientific_name'].replace(['No tree'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Non-sufficient space'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Osage-maclura pomifera'], 'Maclura pomifera')
        data['scientific_name'] = data['scientific_name'].replace(['Other tree'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Palm (unknown'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Palm \(unknown'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Palm evergreen'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Palm sp'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Palm'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Persimmon diospyros'], 'Diospyros')
        data['scientific_name'] = data['scientific_name'].replace(['Persimmon, japanese'], 'Diospyros kaki')
        data['scientific_name'] = data['scientific_name'].replace(['Pheonix robelenii'], 'Phoenix roebelenii')
        data['scientific_name'] = data['scientific_name'].replace(['Pheonix'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Philadelpus'], 'Philadelphus')
        data['scientific_name'] = data['scientific_name'].replace(['Pine tree'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Pistache chinensis'], 'Pistacia chinensis')
        data['scientific_name'] = data['scientific_name'].replace(['Planting site'], '')   
        data['scientific_name'] = data['scientific_name'].replace(['Planting space'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Platanusxacerifolia'], 'Platanus acerifolia')
        # synonyms for the London Plane
        data['scientific_name'] = data['scientific_name'].replace(['Platanus hispanica'], 'Platanus acerifolia')
        data['scientific_name'] = data['scientific_name'].replace(['Platanus hybrida'], 'Platanus acerifolia')
        data['scientific_name'] = data['scientific_name'].replace(['Poor planting'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Potential site'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Preakness pl'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Private shrub'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Privet'], 'Ligustrum')
        data['scientific_name'] = data['scientific_name'].replace(['Queen palm'], 'Syagrus romanzoffiana')
        data['scientific_name'] = data['scientific_name'].replace(['Quercia agrifolia'], 'Quercus agrifolia')
        data['scientific_name'] = data['scientific_name'].replace(['Quercia virginiana'], 'Quercus virginiana')
        data['scientific_name'] = data['scientific_name'].replace(['Raphiolepsis'], 'Rhaphiolepis')
        data['scientific_name'] = data['scientific_name'].replace(['Reference location'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Resident refusal'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Rhodendron'], 'Rhododendron sp.')
        data['scientific_name'] = data['scientific_name'].replace(['Rose-of-hibiscus syriacus'], 'Hibiscus syriacus')
        data['scientific_name'] = data['scientific_name'].replace(['Secretariat dr'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Shrub shrub'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Shrub'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Stump -'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Stump'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Taxas sp'], 'Taxas')
        data['scientific_name'] = data['scientific_name'].replace(['Taxas'], 'Taxus')
        data['scientific_name'] = data['scientific_name'].replace(['Tbd'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Tibuchnia'], 'Tibouchina')
        data['scientific_name'] = data['scientific_name'].replace(['Tili'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Tree(s)'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Tree\(s\)'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Unidientified'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Unsuitable site'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Vacant -'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Vacant planting'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Vacant site'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Vacant site/inadequate'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Vacant site/ok'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Vacant site/prop.'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Vacant well'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Vacant'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Vacant/inadequate spacing'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Vacant/ok to'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Vacant/prop. owner'], '')
        data['scientific_name'] = data['scientific_name'].replace(['W-no sewer/water'], '')
        data['scientific_name'] = data['scientific_name'].replace(['W-no sewer\/water'], '')
        data['scientific_name'] = data['scientific_name'].replace(['W-obstruction at'], '')
        data['scientific_name'] = data['scientific_name'].replace(['W-property owner'], '')
        data['scientific_name'] = data['scientific_name'].replace(['W\-no sewer/water'], '')
        data['scientific_name'] = data['scientific_name'].replace(['W\-ouction at'], '')
        data['scientific_name'] = data['scientific_name'].replace(['Walnut tree'], 'Juglans')
        data['scientific_name'] = data['scientific_name'].replace(['Xancotheras sorbifolium'], 'Xanthoceras sorbifolium')
        data['scientific_name'] = data['scientific_name'].replace(['Yellow-callitropsis nootkatensis'], 'Callitropsis nootkatensis')
        data['scientific_name'] = data['scientific_name'].replace(['\)amelanchier canadensis'], 'Amelanchier canadensis')
        data['scientific_name'] = data['scientific_name'].replace(['\)carpinus caroliniana'], 'Carpinus caroliniana')
        data['scientific_name'] = data['scientific_name'].replace(['\_\_planting request'], '')
        data['scientific_name'] = data['scientific_name'].replace(['__planting request'], '')
        data['scientific_name'] = data['scientific_name'].replace(['_do not'], '')
        data['scientific_name'] = data['scientific_name'].replace(['_multiple species'], '')
        data['scientific_name'] = data['scientific_name'].replace(['_multiple'], '')
        data['scientific_name'] = data['scientific_name'].replace(['_planting space'], '') 
        ## find genus only rows
        # data['genus_only']="full"
        # data['genus_only'][data['scientific_name'].str.contains(' sp.',na=False)] = "genus_only"
#############################
###  Common Name       
#############################
    if 'common_name' in data.columns:
        data['common_name_old']=data['common_name']
        ## remove non ASCII characters
        data['common_name'] = data['common_name'].str.encode('ascii', 'ignore').str.decode('ascii')
        ## quick fix to capitalization issues
        data['common_name']=data['common_name'].str.capitalize()
        ## remove cultivar names Maple 'freeman variety'
        data['common_name']=data['common_name'].str.replace(r'\'.*?\'', '')
        ## get rid of leading spaces
        data['common_name']=data['common_name'].str.strip()
        ## remove SPP issues
        data['common_name'] = data['common_name'].str.replace(" spp.", '')
        data['common_name'] = data['common_name'].str.replace("spp. ", '')
        data['common_name'] = data['common_name'].str.replace("spp", '')
        ## Remove random meaningless information
        data['common_name'] = data['common_name'].str.replace("Other", '')
        data['common_name'] = data['common_name'].str.replace("Columnar ", '')
        data['common_name'] = data['common_name'].str.replace("Various", '')
        data['common_name'] = data['common_name'].str.replace("Unknown", '')
        data['common_name'] = data['common_name'].str.replace("Unknown 1", '')
        data['common_name'] = data['common_name'].str.replace("Unknown other", '')
        data['common_name'] = data['common_name'].str.replace("Unkns", '')
        data['common_name'] = data['common_name'].str.replace("Unknt", '')
        data['common_name'] = data['common_name'].str.replace("Bdl ot", '')
        data['common_name'] = data['common_name'].str.replace("Bdm ot", '')
        data['common_name'] = data['common_name'].str.replace("Bds ot", '')
        data['common_name'] = data['common_name'].str.replace("Nrwy", 'Norway')
        ## fix Maple, Red and "Maple: Red"
        data['common_name'] = data['common_name'].str.split(', ').str[::-1].str.join(' ')
        data['common_name'] = data['common_name'].str.split(': ').str[::-1].str.join(' ')
        data['common_name'] = data['common_name'].str.split('; ').str[::-1].str.join(' ')
        data['common_name'] = data['common_name'].str.split('-').str[::-1].str.join(' ')
        data['common_name'] = data['common_name'].str.split(';').str[::-1].str.join(' ')
        ## remove random commas, dashes, and semicolons
        data['common_name'] = data['common_name'].str.replace(",", '')
        data['common_name'] = data['common_name'].str.replace(";", '')
        ## redo capitalization
        data['common_name']=data['common_name'].str.capitalize()
        ## redo get rid of leading spaces
        data['common_name']=data['common_name'].str.strip()
#############################
###  Address Issues       
#############################
    if 'address' not in data.columns:
        if 'street_name' in data.columns:
            if 'street_number' in data.columns:
                data['address']=data['street_number'].astype(str).str.replace('\.0', '') + " " + data["street_name"].str.strip().str.title().str.replace('\/', '')    
            if 'street_number' not in data.columns:
                data['address']=data["street_name"].str.strip().str.title().str.replace('\/', '')
    if 'address' in data.columns:
        data['address_old']=data['address']
        data['address']=data['address'].str.title().str.replace('Th ', 'th ').str.replace('Rd ', 'rd ')
        data['address']=data['address'].str.replace('^0+','', regex=True)
        data['address']=data['address'].str.replace('^Nan ','', regex=True)
        data['address']=data['address'].str.replace('\s+', ' ', regex=True)
    
#############################
###  Overhead Utility       
#############################
    if 'overhead_utility' in data.columns:
        data['overhead_utility_old'] = data['overhead_utility']
        data['overhead_utility'] = data['overhead_utility'].str.replace('None', 'No')
        data['overhead_utility'] = data['overhead_utility'].str.replace('No Lines', 'No')
        data['overhead_utility'] = data['overhead_utility'].str.replace('No Overhead Utilities', 'No')
        data['overhead_utility'] = data['overhead_utility'].str.replace('Both', 'Yes')
        data['overhead_utility'] = data['overhead_utility'].str.replace('Electric', 'Yes')
        data['overhead_utility'] = data['overhead_utility'].str.replace('Communication', 'Yes')
        data['overhead_utility'] = data['overhead_utility'].str.replace('Present and Conflicting', 'Yes')
        data['overhead_utility'] = data['overhead_utility'].str.replace('Present \/ No Conflict', 'Yes')
        data['overhead_utility'] = data['overhead_utility'].str.replace('Present', 'Yes')
        data['overhead_utility'] = data['overhead_utility'].str.replace('Closer than 20 ft', 'Yes')
        data['overhead_utility'] = data['overhead_utility'].str.replace('Conflicting', 'Yes')
        data['overhead_utility'] = data['overhead_utility'].str.replace('Both', 'Yes')
        data['overhead_utility'] = data['overhead_utility'].str.replace('Low Voltage', 'Yes')
        data['overhead_utility'] = data['overhead_utility'].str.replace('High Voltage', 'Yes')
        data['overhead_utility'] = data['overhead_utility'].str.replace('Unknown','')
#############################
### columns that just need simple capitalization fixes, etc
#############################
    for my_column in ['location_type','location_name',
                      'neighborhood','district','planter_type','ward']:
        if my_column in data.columns:
            ## save original formatting
            data[my_column+'_old']=data[my_column]
            if data[my_column].dtype !="float64":
                if data[my_column].dtype !="int64":
                    ## remove non ASCII characters
                    data[my_column] = data[my_column].str.encode('ascii', 'ignore').str.decode('ascii')
                    ## quick fix to capitalization issues
                    data[my_column]=data[my_column].str.capitalize()
                    ## remove leading and trailing spaces
                    data[my_column]=data[my_column].str.strip()
#############################
### height convert FT to M
#############################
    if "height_FEET" in data.columns: 
        data['height_M'] = data['height_FEET']*3.28084
#############################
### standardize height range
#############################
    if "height_range" in data.columns:
        data['height_range'] = data['height_range'].str.replace('15\-30','15 to 30 ft')
        data['height_range'] = data['height_range'].str.replace('\-\-\-','')
        data['height_range'] = data['height_range'].str.replace('01\-15','0 to 15 ft')
        data['height_range'] = data['height_range'].str.replace('30\-45','30 to 45 ft')
        data['height_range'] = data['height_range'].str.replace('45\-60','45 to 60 ft')
        data['height_range'] = data['height_range'].str.replace('60\+','more than 60 ft')
        data['height_range'] = data['height_range'].str.replace('LT 10\'','0 to 10 ft')
        data['height_range'] = data['height_range'].str.replace('11\-20\'','11 to 20 ft')
        data['height_range'] = data['height_range'].str.replace('21\-30\'','21 to 30 ft')
        data['height_range'] = data['height_range'].str.replace('31\-40\'','31 to 40 ft')
        data['height_range'] = data['height_range'].str.replace('41\-50\'','41 to 50 ft')
        data['height_range'] = data['height_range'].str.replace('GT 50\'','more than 50 ft')
        data['height_range'] = data['height_range'].str.replace('0\-20','0 to 20 ft')
        data['height_range'] = data['height_range'].str.replace('20\-40','20 to 40 ft')
        data['height_range'] = data['height_range'].str.replace('60\-80','60 to 80 ft')
        data['height_range'] = data['height_range'].str.replace('40\-60','40 to 60 ft')
        data['height_range'] = data['height_range'].str.replace('80\+','more than 80 ft')
        data['height_range'] = data['height_range'].str.replace('0ft\-15ft','0 to 15 ft')
        data['height_range'] = data['height_range'].str.replace('30ft\-60ft','30 to 60 ft')
        data['height_range'] = data['height_range'].str.replace('15ft\-30ft','15 to 30 ft')
        data['height_range'] = data['height_range'].str.replace('60ft\+','more than 60 ft')
        data['height_range'] = data['height_range'].str.replace('Over 25 ft','more than 25 ft')
        data['height_range'] = data['height_range'].str.replace('15 \- 25 ft','15 to 25 ft')
        data['height_range'] = data['height_range'].str.replace('6 \- 15 ft','6 to 15 ft')
        data['height_range'] = data['height_range'].str.replace('Under 6 ft','less than 6 ft')
        data['height_range'] = data['height_range'].str.replace('under 25\'','less than 25 ft')
        data['height_range'] = data['height_range'].str.replace('25\' \- 45\'','25 to 45 ft')
        data['height_range'] = data['height_range'].str.replace('over 45\'','more than 45 ft')
        data['height_range'] = data['height_range'].str.replace('20-50','20 to 50 ft')
        data['height_range'] = data['height_range'].str.replace('50 \+','more than 50 ft')
        data['height_range']=data['height_range'].str.strip()
#############################
### bin heights
#############################
    if 'height_M' not in data.columns:
        if 'height_range' in data.columns:
            data['height_binned_M'] = data['height_range']
            data['height_binned_M'] = data['height_binned_M'].replace(['0 to 15 ft'],'0 to 4.57 m')
            data['height_binned_M'] = data['height_binned_M'].replace(['15 to 30 ft'], '4.57 to 9.14 m')
            data['height_binned_M'] = data['height_binned_M'].replace(['30 to 45 ft'], '9.14 to 13.71 m')
            data['height_binned_M'] = data['height_binned_M'].replace(['45 to 60 ft'], '13.71 to 18.29 m')
            data['height_binned_M'] = data['height_binned_M'].replace(['more than 60 ft'], 'more than 18.29 m')
    if 'height_M' in data.columns:    
        data.loc[(data.height_M>0)&(data.height_M<=4.57),'height_binned_M']='0 to 4.57 m'
        data.loc[(data.height_M>4.57)&(data.height_M<=9.14),'height_binned_M']='4.57 to 9.14 m'
        data.loc[(data.height_M>9.14)&(data.height_M<=13.71),'height_binned_M']='9.14 to 13.71 m'
        data.loc[(data.height_M>13.71)&(data.height_M<=18.29),'height_binned_M']='13.71 to 18.29 m'
        data.loc[(data.height_M>18.29),'height_binned_M']='more than 18.29 m'
    ## filter by the good bins
    good_height_bins = ['0 to 4.57 m', '4.57 to 9.14 m', '9.14 to 13.71 m','13.71 to 18.29 m','more than 18.29 m']
    if 'height_binned_M' in data.columns:  
        data['height_binned_M']=data['height_binned_M'].str.strip()
        data.loc[~(data.height_binned_M.isin(good_height_bins)),'height_binned_M']=''       
#############################
### diam breast height convert IN to CM
#############################
    if "diameter_breast_height_IN" in data.columns: 
        data['diameter_breast_height_CM'] = data['diameter_breast_height_IN']*2.54   
#############################
### diam breast height range
#############################
    if 'diameter_breast_height_range' in data.columns:
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('0-3in', '0 to 3 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('12-18in', '12 to 18 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('18-24in', '18 to 24 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('24-30in', '24 to 30 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('3-6in', '3 to 6 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('6-12in', '6 to 12 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('>30in', 'more than 30 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('0-6in', '0 to 6 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('0-15', '0 to 15 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('16-30', '16 to 30 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('31-45', '31 to 45 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('46-60', '46 to 60 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('>60', 'more than 60 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('12 - 24 in', '12 to 24 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('3 - 6 in', '3 to 6 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('6 - 12 in', '6 to 12 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('Over 24 in', 'more than 24 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].str.replace('Under 3 in', '0 to 3 in')
        data['diameter_breast_height_range'] = data['diameter_breast_height_range'].replace(['6 to 12'], '6 to 12 in')
#############################
### diam breast height binned
#############################
    if 'diameter_breast_height_CM' not in data.columns:
        if 'diameter_breast_height_range' in data.columns:
            data['diameter_breast_height_binned'] = data['diameter_breast_height_range']
            data['diameter_breast_height_binned'] = data['diameter_breast_height_binned'].replace(['7 to 12 in'], '6 to 12 in')
            data['diameter_breast_height_binned'] = data['diameter_breast_height_binned'].replace(['0 to 3 in','3 to 6 in'], '0 to 6 in')
            data['diameter_breast_height_binned'] = data['diameter_breast_height_binned'].replace(['13 to 18 in'], '12 to 18 in')
            data['diameter_breast_height_binned'] = data['diameter_breast_height_binned'].replace(['19 to 24 in'], '18 to 24 in')
            data['diameter_breast_height_binned'] = data['diameter_breast_height_binned'].replace(['25 to 30 in'], '24 to 30 in')
            data['diameter_breast_height_binned'] = data['diameter_breast_height_binned'].replace(['31 to 36 in', '37 to 42 in', '43 to 48 in', 'more than 49 in', 
                                                                                                   '30 to 36 in', '36 to 42 in', '42 to 48 in','more than 48 in', 
                                                                                                   'more than 42 in', 'more than 36 in'], 'more than 30 in')
    ## make a CM column
            data['diameter_breast_height_binned_CM'] = data['diameter_breast_height_binned']
            data['diameter_breast_height_binned_CM'] = data['diameter_breast_height_binned_CM'].replace(['0 to 6 in'], '0 to 15.24 cm')
            data['diameter_breast_height_binned_CM'] = data['diameter_breast_height_binned_CM'].replace(['6 to 12 in'], '15.24 to 30.48 cm')
            data['diameter_breast_height_binned_CM'] = data['diameter_breast_height_binned_CM'].replace(['12 to 18 in'], '30.48 to 45.72 cm')
            data['diameter_breast_height_binned_CM'] = data['diameter_breast_height_binned_CM'].replace(['18 to 24 in'], '45.72 to 60.96 cm')
            data['diameter_breast_height_binned_CM'] = data['diameter_breast_height_binned_CM'].replace(['24 to 30 in'], '60.96 to 76.2 cm')
            data['diameter_breast_height_binned_CM'] = data['diameter_breast_height_binned_CM'].replace(['more than 30 in'], 'more than 76.2 cm')
    if 'diameter_breast_height_CM' in data.columns:    
        data.loc[(data.diameter_breast_height_CM>0)&(data.diameter_breast_height_CM<=15.24),'diameter_breast_height_binned_CM']='0 to 15.24 cm'
        data.loc[(data.diameter_breast_height_CM>15.24)&(data.diameter_breast_height_CM<=30.48),'diameter_breast_height_binned_CM']='15.24 to 30.48 cm'
        data.loc[(data.diameter_breast_height_CM>30.48)&(data.diameter_breast_height_CM<=45.72),'diameter_breast_height_binned_CM']='30.48 to 45.72 cm'
        data.loc[(data.diameter_breast_height_CM>45.72)&(data.diameter_breast_height_CM<=60.96),'diameter_breast_height_binned_CM']='45.72 to 60.96 cm'
        data.loc[(data.diameter_breast_height_CM>60.96)&(data.diameter_breast_height_CM<=76.2),'diameter_breast_height_binned_CM']='60.96 to 76.2 cm'
        data.loc[(data.diameter_breast_height_CM>76.2),'diameter_breast_height_binned_CM']='more than 76.2 cm'
    ## filter by the good bins
    good_bins = ['0 to 15.24 cm', '15.24 to 30.48 cm', '30.48 to 45.72 cm', '45.72 to 60.96 cm', 
                 '60.96 to 76.2 cm', 'more than 76.2 cm']
    if 'diameter_breast_height_binned_CM' in data.columns:  
        data.loc[~(data.diameter_breast_height_binned_CM.isin(good_bins)),'diameter_breast_height_binned_CM']='' 
#############################
### condition (including alive_status)
#############################        
    if "alive_status" in data.columns:
        data['alive_status'] = data['alive_status'].replace(['Proposed Site - Large'], '')
        data['alive_status'] = data['alive_status'].replace(['Recently Planted'], '')
        data['alive_status'] = data['alive_status'].replace(['Removed'], '')
        data['alive_status'] = data['alive_status'].replace(['Stump'], 'dead')
        data['alive_status'] = data['alive_status'].replace(['False'], 'dead')
        data['alive_status'] = data['alive_status'].replace(['True'], 'alive')
        data['alive_status'] = data['alive_status'].replace(['Dead'], 'dead')
        data['alive_status'] = data['alive_status'].replace(['Alive'], 'alive')
    if "condition" in data.columns:
        data['condition'] = data['condition'].str.capitalize()
        data['condition'] = data['condition'].replace(['Dead', 'Dead 0%'], 'dead')
        data['condition'] = data['condition'].replace(['Dead / dying', 'Dead/dying', 'Dead/nearly dead/planting space/stump', 
                                                       'Dead or dying'], 'dead/dying')
        data['condition'] = data['condition'].replace(['Excellent', 'Very good', 'Excellent / specimen tree', 
                                                       'Excellent 100%', 'Very good 90%'], 'excellent')
        data['condition'] = data['condition'].replace(['Fair - minor problems', 'Fair', 'Fair / generally healthy', 
                                                       'Fair 60%'], 'fair')
        data['condition'] = data['condition'].replace(['Good - no apparent problems', 'Good', 'Healthy', 
                                                       'Good / healthy and vigorous', 'Good 80%'], 'good')
        data['condition'] = data['condition'].replace(['','0','2','3','4','Not Applicable','Not applicable', 
                                                       'Cannot be determined', 'Stump', 'Removal', 'Unknown',
                                                       'Not entered', 'Remove', 'Field chk', 'Not check*', 
                                                       'Vacant', 'Unassigned'], 'NA')
        data['condition'] = data['condition'].replace(['Poor - major problems', 'Poor', 'Poor ','Very Poor','Very poor', 
                                                       'Critical', 'Dying', 'Stressed', 'Poor / declining', 
                                                       'Condition 20%', 'Critical 20%', 'Poor 40%', 'Very poor 30%'], 'poor')

#############################
### date columns
#############################
# cycle through all date-like columns
# get them all into 10/02/1990 format
    for my_date_column in ['planted_date', 'inventory_date', 'condition_date',
                       'edit_date', 'inspected_date', 'retired_date']:
        if my_date_column in data.columns:
            data[my_date_column] = data[my_date_column].replace([0,-1,99], '')
            data[my_date_column] = data[my_date_column].astype(str)
            # remove time from columns that have it, delimited by a T or space
            data.loc[data[my_date_column].str.contains('T'), my_date_column] = data[my_date_column].str.split('T').str[0]
            data.loc[data[my_date_column].str.contains(' '), my_date_column] = data[my_date_column].str.split(' ').str[0]
            # change 2018.0 to 2018
            data.loc[data[my_date_column].str.contains('.'), my_date_column] = data[my_date_column].str.split('.').str[0]
            # reformat 1990-12-2 to 12/02/1990. It is year-month-day format for the cities
            dates_with_dashes = data.loc[data[my_date_column].str.contains('-'), my_date_column]
            year = dates_with_dashes.str.split('-').str[0]
            month = dates_with_dashes.str.split('-').str[1]
            day = dates_with_dashes.str.split('-').str[2]
            new_dates_with_dashes = month + '/' + day +'/'+year
            data.loc[data[my_date_column].str.contains('-'), my_date_column]=new_dates_with_dashes
            # sometimes they format it 2020/09/15. We need to find those cases and reorder.
            dates_with_slashes = data.loc[data[my_date_column].str.contains('/'), my_date_column]
            first = dates_with_slashes.str.split('/').str[0]
            second = dates_with_slashes.str.split('/').str[1]
            third = dates_with_slashes.str.split('/').str[2]
            # is the first value the year?
            count_yearlike = sum(i > 1900 for i in first.value_counts().sort_values(ascending=False).keys().astype(int)[0:10]) 
            # set threshold at 8 out of 10 need to be yearlike, assuming some blanks etc
            if count_yearlike >= 8:
                new_dates_with_slashes = second + '/' + third +'/'+ first
                data.loc[data[my_date_column].str.contains('/'), my_date_column]=new_dates_with_slashes              
# now find the most recent "observation" out of inventory, condition, edit, and inspected
    observation_date_columns = ['inventory_date', 'condition_date','edit_date', 'inspected_date']
    observations = data.loc[:,data.columns.isin(observation_date_columns)]
    number_columns = observations.shape[1]
    if number_columns == 1:
        data['most_recent_observation'] = observations
        data['most_recent_observation_type'] = observations.columns[0]
    if number_columns > 1:
        for c in observations.columns:
            observations[c] = pd.to_datetime(observations[c],errors = 'coerce')
        data['most_recent_observation'] = observations.max(axis=1)
        data['most_recent_observation'] =  data['most_recent_observation'].astype(str)
        ## pd.to_datetime automatically formats it as 2020-03-30, so I change it back here
        observations_with_dashes = data.loc[data['most_recent_observation'].str.contains('-'), 'most_recent_observation']
        year_o = observations_with_dashes.str.split('-').str[0]
        month_o = observations_with_dashes.str.split('-').str[1]
        day_o = observations_with_dashes.str.split('-').str[2]
        new_observations_with_dashes = month_o + '/' + day_o +'/'+year_o
        data.loc[data['most_recent_observation'].str.contains('-'), 'most_recent_observation']=new_observations_with_dashes
#############################
### location_type
#############################            
    if "location_type" in data.columns:
        data['location_type_old'] = data['location_type']
        data['location_type'] = data['location_type'].replace(['999'], 'no_info')
        data['location_type'] = data['location_type'].replace(['\<null\>'], 'no_info')
        data['location_type'] = data['location_type'].replace(['1st row'], 'no_info')
        data['location_type'] = data['location_type'].replace(['2nd row'], 'no_info')
        data['location_type'] = data['location_type'].replace(['3rd row'], 'no_info')
        data['location_type'] = data['location_type'].replace(['4th row'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Across the street'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Arboretum'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Arterial roadway'], '')
        data['location_type'] = data['location_type'].replace(['Arts commission'], '')
        data['location_type'] = data['location_type'].replace(['Asian arts commission'], '')
        data['location_type'] = data['location_type'].replace(['Athletic field'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Back of sidewalk'], '')
        data['location_type'] = data['location_type'].replace(['Backup'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Baseball field'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Basin'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Basketball court'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Building'], '')
        data['location_type'] = data['location_type'].replace(['Business/commercial'], '')
        data['location_type'] = data['location_type'].replace(['Cart path'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Cemetery'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Church / school'], '')
        data['location_type'] = data['location_type'].replace(['City building'], '')
        data['location_type'] = data['location_type'].replace(['City college'], '')
        data['location_type'] = data['location_type'].replace(['City maintained street tree'], '')
        data['location_type'] = data['location_type'].replace(['City owned parcel'], '')
        data['location_type'] = data['location_type'].replace(['City property'], '')
        data['location_type'] = data['location_type'].replace(['Cjots'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Cns'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Cnty'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Contrctr'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Curbtight'], '')
        data['location_type'] = data['location_type'].replace(['Cutout'], '')
        data['location_type'] = data['location_type'].replace(['Denver fire dept'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Dept of real estate'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Disc golf course'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Dog park'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Don'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Downtown'], '')
        data['location_type'] = data['location_type'].replace(['Dpw'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Dpw for city agency'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Driving range'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Easement tree'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Entrance'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Facility'], '')
        data['location_type'] = data['location_type'].replace(['Fairway'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Fence row'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Ffd'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Fire dept'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Golf course'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Golf course rough'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Health dept'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Hlcc'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Hoa street tree'], '')
        data['location_type'] = data['location_type'].replace(['Housing authority'], '')
        data['location_type'] = data['location_type'].replace(['Industrial/ large commercial'], '')
        data['location_type'] = data['location_type'].replace(['Industrial/large commercial'], '')
        data['location_type'] = data['location_type'].replace(['Institutional'], '')
        data['location_type'] = data['location_type'].replace(['Island'], '')
        data['location_type'] = data['location_type'].replace(['Lawn'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Maintenance facilit'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Mayor office of housing'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Median'], '')
        data['location_type'] = data['location_type'].replace(['Mountain park'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Mta'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Multi family'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Multi-family residential'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Natural area'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Ncns'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Not entered'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Nursery'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Office of mayor'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Offsetfromcurb'], '')
        data['location_type'] = data['location_type'].replace(['Oncurb'], '')
        data['location_type'] = data['location_type'].replace(['Open'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Open/unrestricted'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Orphan parcel'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Other'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Other public property'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Other un-maintained locations'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Other/maintained'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Park'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Park strip'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Park/ vacant/ other'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Park/golf course'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Park/greenbelt'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Park/public space'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Park/vacant/other'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Parking area'], '')
        data['location_type'] = data['location_type'].replace(['Parkway'], '')
        data['location_type'] = data['location_type'].replace(['Paseo'], 'built_environment')
        data['location_type'] = data['location_type'].replace(['Paver'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Picnic area'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Pinetum'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Planter'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Playground'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Plaza'], 'built_environment')
        data['location_type'] = data['location_type'].replace(['Police dept'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Pond or creek'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Port'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Pos'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Priv'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Private'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Private maintained street tree'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Private property tree'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Public library'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Puc'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Pump house'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Purchasing dept'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Raised planter'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Rec/park'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Residential private tree'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Residential/commercial'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Right-of-way'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Row 1 ats'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Row1 across street'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Row2 across street'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Row3 across street'], 'no_info')
        data['location_type'] = data['location_type'].replace(['School'], 'green_space')
        data['location_type'] = data['location_type'].replace(['School site'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Scl'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Sdot'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Seac'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Sfusd'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Sidewalk'], 'built_environment')
        data['location_type'] = data['location_type'].replace(['Single family'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Single-family residential'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Small commercial'], 'built_environment')
        data['location_type'] = data['location_type'].replace(['Sndtr'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Sph'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Spray pool'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Sps'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Spu'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Stewardship tree'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Street'], 'built_environment')
        data['location_type'] = data['location_type'].replace(['Street tree'], 'built_environment')
        data['location_type'] = data['location_type'].replace(['Street tree (boulevard or median)'], 'built_environment')
        data['location_type'] = data['location_type'].replace(['Streetscape'], 'built_environment')
        data['location_type'] = data['location_type'].replace(['Strip'], 'built_environment')
        data['location_type'] = data['location_type'].replace(['Swale'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Swimming pool'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Tennis court'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Traffic island'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Trail'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Tree lawn'], 'green_space')
        data['location_type'] = data['location_type'].replace(['Tree well'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Unassigned'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Unmaintained area'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Vacant/other'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Walking path'], 'green_space')
        data['location_type'] = data['location_type'].replace(['War memorial'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Well or pit'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Well/pit'], 'no_info')
        data['location_type'] = data['location_type'].replace(['Woodline'], 'green_space')
        data['location_type'] = data['location_type'].replace([''], 'no_info')           
#############################
#############################
#############################
###
### write CSV
###
#############################
#############################
#############################
    data.to_csv(path + '\\'+ state + '_edited.csv',index=False)

C:\Users\dakot\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\dakot\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (15,18,25,32,38,39,40,41,42,45,50,62,63) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\dakot\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-5-6627c810a27a>:490: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

<ipython-input-5-6627c810a27a>:490: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  observations[c] = pd.to_datetime(observations[c],errors = 'coerce')
<ipython-input-5-6627c810a27a>:490: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  observations[c] = pd.to_datetime(observations[c],errors = 'coerce')
C:\Users\dakot\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (26,27,33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_

In [6]:
## if error see which file is the issue
file

'Worcester_trees.csv'

## Make CSV of all scientific names
To check in R taxize

In [7]:
cd C:\Users\dakot\Documents\Trees\Data Cleaning\Sheets_Data_Misspellings_Corrected

C:\Users\dakot\Documents\Trees\Data Cleaning\Sheets_Data_Misspellings_Corrected


In [8]:
csv_files_edited = glob.glob('*.csv')

get value counts for all

In [9]:
# # set up a "dictionary" for our results
results = {}
for file in csv_files_edited:
   # read the file
   data = pd.read_csv(file)
   # save the state name
   state = file.split('_')[0]
   #if it has scientific names
   if 'scientific_name' in data.columns:
        list=data.groupby(['scientific_name']).size().reset_index().rename(columns={0:'count'})
        results[file] = list
        
# concatenate everything and get good labels
all_scientific=pd.concat(results,axis=0)
species = all_scientific.reset_index()
species = species.drop(['level_1'],axis=1)
species=species.rename(columns={'level_0':'file'})

# print to csv
path=r'C:\Users\dakot\Documents\Trees\Data Cleaning'
species.to_csv(path +'\\' + 'Species_By_City.csv')

## to get total count
species_count = pd.DataFrame(species.groupby(['scientific_name'])['count'].sum().sort_values()).reset_index()
species_count.to_csv(path +'\\' + 'Species_Counts.csv')

C:\Users\dakot\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\dakot\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (15,18,25,32,38,39,40,41,42,45,50,62,63) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\dakot\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\dakot\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (24,25,26,27,28,29,30,31) have mixed types.S

## Get CSV of common names
to send to taxize

In [10]:
# # set up a "dictionary" for our results
results = {}
for file in csv_files_edited:
   # read the file
   data = pd.read_csv(file)
   #if it has common names but no scientific names
   if "scientific_name" not in data.columns:
        if 'common_name' in data.columns:
            list=data.groupby(['common_name']).size().reset_index().rename(columns={0:'count'})
            results[file] = list
        
# concatenate everything and get good labels
all_scientific=pd.concat(results,axis=0)
species = all_scientific.reset_index()
species = species.drop(['level_1'],axis=1)
species=species.rename(columns={'level_0':'city'})

# print to csv
path=r'C:\Users\dakot\Documents\Trees\Data Cleaning'
species.to_csv(path +'\\' + 'Common_Names_By_City.csv')

## to get total count
species_count = pd.DataFrame(species.groupby(['common_name'])['count'].sum().sort_values()).reset_index()
species_count.to_csv(path +'\\' + 'Common_Names_Counts.csv')

# Get CSV of all city and state names

In [11]:
results = {}
for file in csv_files_edited:
   # read the file
   data = pd.read_csv(file)
   # save the city name
   city = file.split('_')[0]
   #if it has common names but no scientific names
   list=data.groupby(['city','state']).size().reset_index().rename(columns={0:'count'})
   results[file] = list
            
# concatenate everything and get good labels
all_cities=pd.concat(results,axis=0).reset_index().drop(['level_1'],axis=1).rename(columns={'level_0':'filename'})
# print to csv
path=r'C:\Users\dakot\Documents\Trees\Data Cleaning'
all_cities.to_csv(path +'\\' + 'Cities.csv')